In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=830752960536-ibup9rkb1p8h2v4ig8csiav3f0nqol6m.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [17]:
import os
import time
from datetime import datetime
import requests

In [3]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "CZ3002 Pictures"):
      fileID = file['id']

Title: CZ3002 Pictures, ID: 1cCb0a9gKnRMP035Ts_q8Y2P5Kfhk2yQ_
Title: Untitled Diagram.drawio, ID: 1SC_KQHEbkVyYYLk9lFjZUZ3LcEfxRe8v
Title: 项目计划书, ID: 1TezH9Tsdn7ee_v5KXl6XePa4DUw5pPiPgXIARU9dGYE
Title: Relational Tables, ID: 1DLfvoD67lh6C611RFvzM1n8C7L6XtdTd2c0mGnRsgDw
Title: Table Tennis Animation.unitypackage, ID: 1dM_WHKJaVMApmxnqjrvyPuhLBVKswHfk
Title: 简历, ID: 1poRTn79u7RaUe3s36g-OLnEiswg5qanLktXknvfdI8w
Title: 无标题文档, ID: 1_d9iKeRidfGbAtf7UKws4gtEWvDFVdkfadJVmzo09bk
Title: Untitled Diagram.drawio, ID: 1JuC7MsgXGWu_8hOYaYYB9HwCVGUr-GhT
Title: Audience Analysis, ID: 1dtGYf1eoaBDWo52c9-rEO_WgSyGiV5VpZALXLsl1-EY
Title: TP, ID: 1dvTgnTVL3mN7G1XfnRhgKF1U73JZte6BJKXaVgSTY5M
Title: NN/g Customer Journey Mapping Template（副本）, ID: 1CF2sgtqbWeQGrTN_Tm6HQqzBlVplGgAPQUkdNlD22UU
Title: IET_student_application.pdf, ID: 1diQ-TtTxEgLjVNydoVONeM0uZnw6v2Qv
Title: IMG_4593.PNG, ID: 16wfZJ3ej5ZXNxkwaUDi3479cCR2HUFgC
Title: 无标题文档, ID: 1pDztY5c_EM_MH0iLvM44BNjiK-qBuJvJ5vtHnmzHZdY
Title: Colab Notebooks, 

In [4]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("C:\\Users\\User\Downloads\cz3002-33787-firebase-adminsdk-mu28c-ef5af6f8bf.json")
firebase_admin.initialize_app(cred)

In [5]:
fileArr = os.listdir()
def checkDifference():
    global fileArr
    previousFileArr = fileArr
    fileArr = os.listdir()
    difference = set(fileArr)-set(previousFileArr)
    fileArr = os.listdir()
    print("Difference check successful")
    return difference

In [6]:
def jpgAutoCehckAndUpload():
    difference = checkDifference()
    if(difference == set()):
        print("There is no difference")
        return False
    else:
        for i in difference:
            fileUp = drive.CreateFile({"mimeType": "image/jpeg", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
            fileUp.SetContentFile(i)
            fileUp.Upload() # Upload the file.
            print('Created file %s with mimeType %s' % (fileUp['title'], fileUp['mimeType'])) 
            now = datetime.now()
            return [fileUp['id'],now]

In [7]:
flag = 0
while(flag!=10):
    time.sleep(2)
    result = jpgAutoCehckAndUpload()
    print(result)
    if(result != False):
        Pictures_Rec(result[0],result[1]).write()
    flag = flag+1

Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False
Difference check successful
There is no difference
False


In [8]:
from firebase_admin import firestore
db = firestore.client()

In [9]:
Pictures_Rec_ref = db.collection(u'Pictures')
class Pictures_Rec(object):
    index = -1
    def __init__(self,PictureGID,Time):
        self.PictureGID = PictureGID
        self.Time = Time
        Pictures_Rec.index +=1
    def to_dict(self):
        return({
            "PictureGID":self.PictureGID,
            "Time":self.Time
        }
        )
    def write(self):
        Pictures_Rec_ref.document(str(Pictures_Rec.index)).set(self.to_dict())
    @staticmethod
    def from_dict(d):
        return(
            Pictures_Rec(d['PictureGID'],d['Time'])
        )
    def reset():
        Pictures_Rec.index = -1
    def update(doc, key, value):
        db.collection(u'Pictures').document(str(doc)).update({key: value})
    def read_to_dic(doc):
        return(
            db.collection(u'Pictures').document(str(doc)).get().to_dict()
        )
    def read_to_cla(doc):
        return(
            Pictures_Rec.from_dict(db.collection(u'Pictures').document(str(doc)).get().to_dict())
        )

In [10]:
Picture = Pictures_Rec.read_to_dic(0)

In [13]:
PictureGID = Picture['PictureGID']

In [18]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [24]:
download_file_from_google_drive(PictureGID,"C:\\Users\\User\Downloads\D.jpeg")